![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use `JOIN` to merge tables along logical columns
    - Grouping Data with SQL, inlcuding `HAVING`
- Goal 5: Convert SQL to pandas

## Goal 1: Summarize

To date the data we've seen has looked like [this.](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm)

That is how we need data to look to run analysis and build models.<br>
But it doesn't _live_ there in it's native state.

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

## What does it mean to **Engineer Data**?

Let's start with a basic scenario:<br>
You are HR. You have *no* current database. 
How would you set it up?

Data needs considerations:
- What you want to store
- What "views" you anticipate wanting in the future

Structure considerations:
- Speed of retrieval
- How much data you are accessing
- How much you are storing

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
### SQL

[History of SQL](https://www.businessnewsdaily.com/5804-what-is-sql.html)

> Future users of large data banks must be protected from having to know how the data is organized in the machine (the internal representation). A prompting service which supplies such information is not a satisfactory solution. Activities of users at terminals and most application programs should remain unaffected when the internal representation of data is changed and even when some aspects of the external representation are changed. Changes in data representation will often be needed as a result of changes in query, update, and report traffic and natural growth in the types of stored information.
.
.
.<br>
<br>
In Section 1 a relational model of data is proposed as a basis for protecting users of formatted data systems from the potentially disruptive changes in data representation caused by growth in the data bank and changes in traffic.

[The original SQL paper from the 1970s.](https://www.seas.upenn.edu/~zives/03f/cis550/codd.pdf)

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

***
### SQLite

<img src="img/SQLite-Python.jpg" width=500>

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to other database types there are many different libraries:
- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

These all implement [PEP 249: DB API v2](https://www.python.org/dev/peps/pep-0249/)

### The Structure of a SQL Query

<img src='img/sql_statement.jpg'/>

***
## Goal 3: Get going with sqlite!

In [1]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [2]:
con = sqlite3.connect('data/flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [3]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


In [4]:
cursor.description

(('name', None, None, None, None, None, None),)

In [5]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(cursor.fetchall())

[('airports', 'table'), ('ix_airports_index', 'index'), ('airlines', 'table'), ('ix_airlines_index', 'index'), ('routes', 'table'), ('ix_routes_index', 'index')]


In [6]:
cursor.execute("SELECT * FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 'airports', 2, 'CREATE TABLE airports (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [city] TEXT,\n  [country] TEXT,\n  [code] TEXT,\n  [icao] TEXT,\n  [latitude] TEXT,\n  [longitude] TEXT,\n  [altitude] TEXT,\n  [offset] TEXT,\n  [dst] TEXT,\n  [timezone] TEXT\n)')
('index', 'ix_airports_index', 'airports', 3, 'CREATE INDEX ix_airports_index ON airports ([index])')
('table', 'airlines', 'airlines', 945, 'CREATE TABLE airlines (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [alias] TEXT,\n  [iata] TEXT,\n  [icao] TEXT,\n  [callsign] TEXT,\n  [country] TEXT,\n  [active] TEXT\n)')
('index', 'ix_airlines_index', 'airlines', 946, 'CREATE INDEX ix_airlines_index ON airlines ([index])')
('table', 'routes', 'routes', 1393, 'CREATE TABLE routes (\n[index] INTEGER,\n  [airline] TEXT,\n  [airline_id] TEXT,\n  [source] TEXT,\n  [source_id] TEXT,\n  [dest] TEXT,\n  [dest_id] TEXT,\n  [codeshare] TEXT,\n  [stops] TEXT,\n  [equipment] TEXT\n)')
('index', 'ix_routes_i

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [7]:
cursor.execute("SELECT * FROM airports").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [8]:
cursor.fetchone()

(1,
 '2',
 'Madang',
 'Madang',
 'Papua New Guinea',
 'MAG',
 'AYMD',
 '-5.207083',
 '145.7887',
 '20',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [9]:
cursor.fetchmany(4)

[(2,
  '3',
  'Mount Hagen',
  'Mount Hagen',
  'Papua New Guinea',
  'HGU',
  'AYMH',
  '-5.826789',
  '144.295861',
  '5388',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (3,
  '4',
  'Nadzab',
  'Nadzab',
  'Papua New Guinea',
  'LAE',
  'AYNZ',
  '-6.569828',
  '146.726242',
  '239',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (4,
  '5',
  'Port Moresby Jacksons Intl',
  'Port Moresby',
  'Papua New Guinea',
  'POM',
  'AYPY',
  '-9.443383',
  '147.22005',
  '146',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (5,
  '6',
  'Wewak Intl',
  'Wewak',
  'Papua New Guinea',
  'WWK',
  'AYWK',
  '-3.583828',
  '143.669186',
  '19',
  '10',
  'U',
  'Pacific/Port_Moresby')]

#### Use description

In [10]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [11]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
info

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'id', 'TEXT', 0, None, 0),
 (2, 'name', 'TEXT', 0, None, 0),
 (3, 'city', 'TEXT', 0, None, 0),
 (4, 'country', 'TEXT', 0, None, 0),
 (5, 'code', 'TEXT', 0, None, 0),
 (6, 'icao', 'TEXT', 0, None, 0),
 (7, 'latitude', 'TEXT', 0, None, 0),
 (8, 'longitude', 'TEXT', 0, None, 0),
 (9, 'altitude', 'TEXT', 0, None, 0),
 (10, 'offset', 'TEXT', 0, None, 0),
 (11, 'dst', 'TEXT', 0, None, 0),
 (12, 'timezone', 'TEXT', 0, None, 0)]

#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [12]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
info

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'id', 'TEXT', 0, None, 0),
 (2, 'name', 'TEXT', 0, None, 0),
 (3, 'alias', 'TEXT', 0, None, 0),
 (4, 'iata', 'TEXT', 0, None, 0),
 (5, 'icao', 'TEXT', 0, None, 0),
 (6, 'callsign', 'TEXT', 0, None, 0),
 (7, 'country', 'TEXT', 0, None, 0),
 (8, 'active', 'TEXT', 0, None, 0)]

In [13]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
info

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'airline', 'TEXT', 0, None, 0),
 (2, 'airline_id', 'TEXT', 0, None, 0),
 (3, 'source', 'TEXT', 0, None, 0),
 (4, 'source_id', 'TEXT', 0, None, 0),
 (5, 'dest', 'TEXT', 0, None, 0),
 (6, 'dest_id', 'TEXT', 0, None, 0),
 (7, 'codeshare', 'TEXT', 0, None, 0),
 (8, 'stops', 'TEXT', 0, None, 0),
 (9, 'equipment', 'TEXT', 0, None, 0)]

***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [14]:
cursor.execute("""
SELECT *
FROM airlines
""")
cursor.fetchmany(5)

[(0, '1', 'Private flight', '\\N', '-', None, None, None, 'Y'),
 (1, '2', '135 Airways', '\\N', None, 'GNL', 'GENERAL', 'United States', 'N'),
 (2, '3', '1Time Airline', '\\N', '1T', 'RNX', 'NEXTIME', 'South Africa', 'Y'),
 (3,
  '4',
  '2 Sqn No 1 Elementary Flying Training School',
  '\\N',
  None,
  'WYT',
  None,
  'United Kingdom',
  'N'),
 (4, '5', '213 Flight Unit', '\\N', None, 'TFU', None, 'Russia', 'N')]

In [15]:
cursor.execute("""
SELECT name
FROM airlines
WHERE active = 'Y'
  AND country = 'United Kingdom';
""")
cursor.fetchmany(5)

[('Astraeus',),
 ('Air Southwest',),
 ('Aurigny Air Services',),
 ('Air Wales',),
 ('AD Aviation',)]

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

<img src="img/venn.png" width=550>

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [16]:
# used left join and joined on code column from airports and dest column from routes
cursor.execute("""

SELECT R.*, A.latitude, A.longitude
FROM routes R
LEFT JOIN airports A ON A.code = R.dest
LIMIT 5

""")
print(*cursor.fetchall(), sep='\n')

(0, '2B', '410', 'AER', '2965', 'KZN', '2990', None, '0', 'CR2', '55.606186', '49.278728')
(1, '2B', '410', 'ASF', '2966', 'KZN', '2990', None, '0', 'CR2', '55.606186', '49.278728')
(2, '2B', '410', 'ASF', '2966', 'MRV', '2962', None, '0', 'CR2', '44.225072', '43.081889')
(3, '2B', '410', 'CEK', '2968', 'KZN', '2990', None, '0', 'CR2', '55.606186', '49.278728')
(4, '2B', '410', 'CEK', '2968', 'OVB', '4078', None, '0', 'CR2', '55.012622', '82.650656')


### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [17]:
# Which countries have the highest amount of active airports?
#printing top 5 active airports
cursor.execute("""
SELECT country, count(active)
FROM airlines
GROUP BY country
HAVING active='Y'
ORDER BY count(active) desc
LIMIT 5
""")
print(*cursor.fetchall(), sep='\n')

('France', 119)
('Australia', 93)
('South Africa', 91)
('Italy', 90)
('Switzerland', 60)


In [18]:
# Which countries have the highest amount of inactive airports?
# priting top 5 inactive airports
cursor.execute("""
SELECT country, count(active)
FROM airlines
GROUP BY country
HAVING active='N'
ORDER BY count(active) desc
LIMIT 5
""")
print(*cursor.fetchall(), sep='\n')

('United States', 1080)
('Mexico', 439)
('United Kingdom', 407)
('Canada', 318)
('Russia', 230)


In [19]:
# What about airports by timezones?
# printing top 5 airports by timezone
cursor.execute("""
SELECT timezone,count(name)
FROM airports
GROUP BY timezone
ORDER BY count(name) desc
LIMIT 5
""")
print(*cursor.fetchall(), sep='\n')

('America/New_York', 628)
('America/Chicago', 373)
('Europe/Berlin', 319)
('America/Anchorage', 258)
('Europe/Paris', 232)


In [20]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

## Goal 5: Transfering from sqlite to pandas

In [21]:
import pandas as pd

pd_con = sqlite3.connect("data/flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [22]:
# Which countries have the highest amount of active airports?
sql = '''
SELECT country, count(active)
FROM airlines
GROUP BY country
HAVING active='Y'
ORDER BY count(active) desc
LIMIT 5
'''
pd.read_sql_query(sql, pd_con)

,country,count(active)
0,France,119
1,Australia,93
2,South Africa,91
3,Italy,90
4,Switzerland,60


In [23]:
pd_con.close()

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [24]:
conn = sqlite3.connect('data/Chinook_Sqlite.sqlite')
cur = conn.cursor()

In [25]:
# How many tables are in the database?
# printing count of tables from chinook_sqlite database
table = "SELECT name FROM sqlite_master WHERE type='table';"
df = pd.read_sql_query(table,conn)
df.count()

name    11
dtype: int64

In [26]:
# What's the primary key of each table?
# using pragma_table_info printed primary keys of all tables and assigned them to a dataframe
df_pk = pd.DataFrame()
rows = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")
tables = [row[0] for row in rows]
list_pk = []
def sql_identifier(s):
    return '"' + s.replace('"', '""') + '"'
for table in tables:
    rows = cur.execute("SELECT name FROM pragma_table_info({}) WHERE pk = 1".format(sql_identifier(table)))
    list_pk.append(list(rows.fetchall()[0])[0])
df_pk['Table_Name'] = tables
df_pk['PK'] = list_pk
df_pk

,Table_Name,PK
0,Album,AlbumId
1,Artist,ArtistId
2,Customer,CustomerId
3,Employee,EmployeeId
4,Genre,GenreId
5,Invoice,InvoiceId
6,InvoiceLine,InvoiceLineId
7,MediaType,MediaTypeId
8,Playlist,PlaylistId
9,PlaylistTrack,PlaylistId


In [27]:
# What foreign keys join the tables together?
# used pragma foreign_key_list to get the foreign keys data
df_fk = pd.DataFrame()
list_fk = []
for table in tables:
    rows = cur.execute("PRAGMA foreign_key_list({})".format(sql_identifier(table)))
    list_temp = []
    temp_str = ''
    list_temp = list(rows.fetchall())
    for i in range(len(list_temp)):
            temp_str = temp_str + list_temp[i][3] + ','
    list_fk.append(temp_str.strip(','))
df_fk['Table_Name'] = tables
df_fk['FK'] = list_fk
df_fk
# hint: use "PRAGMA foreign_key_list()"


,Table_Name,FK
0,Album,ArtistId
1,Artist,
2,Customer,SupportRepId
3,Employee,ReportsTo
4,Genre,
5,Invoice,CustomerId
6,InvoiceLine,"TrackId,InvoiceId"
7,MediaType,
8,Playlist,
9,PlaylistTrack,"TrackId,PlaylistId"


In [28]:
# What are the max and min dates in the Invoice table?
# printing max and min dates as datetime
sql = '''
SELECT MAX(InvoiceDate) as max_date, MIN(InvoiceDate) as min_date 
FROM Invoice
'''
pd.read_sql_query(sql, conn)

,max_date,min_date
0,2013-12-22 00:00:00,2009-01-01 00:00:00


In [29]:
# What tables would you need to answer "what is your most popular track?"
# PlaylistTrack, Track

In [30]:
# What values from each table?
# TrackID, TrackName

In [31]:
# Put it all together:
# You need to create a query that can rank tracks in term of popularity.

# printing trackid's based on popularity by joining playlisttrack and track tables
sql = '''
SELECT P.TrackId, T.Name, count(P.TrackId) as Popularity
FROM PlaylistTrack P join Track T on P.TrackId = T.TrackId
GROUP BY P.TrackId 
ORDER BY Popularity desc
'''
pd.read_sql_query(sql, conn)

,TrackId,Name,Popularity
0,3403,Intoitus: Adorate Deum,5
1,3404,"Miserere mei, Deus",5
2,3408,"Aria Mit 30 Veränderungen, BWV 988 ""Goldberg V...",5
3,3409,"Suite for Solo Cello No. 1 in G Major, BWV 100...",5
4,3410,"The Messiah: Behold, I Tell You a Mystery... T...",5
...,...,...,...
3498,3474,October Song,2
3499,3475,What Is It About Men,2
3500,3476,Help Yourself,2
3501,3477,Amy Amy Amy (Outro),2


In [32]:
# Advanced: get the artist who sang the song!

# printing artistname, trackname and trackid by joining artist, album and track tables 
sql = '''
SELECT A.Name as Artist_Name, T.Name as Track_Name, T.TrackID
FROM Artist A join Album ALB on A.ArtistID = ALB.ArtistID
JOIN Track T on ALB.AlbumID = T.AlbumID
'''
pd.read_sql_query(sql, conn)

,Artist_Name,Track_Name,TrackId
0,AC/DC,For Those About To Rock (We Salute You),1
1,Accept,Balls to the Wall,2
2,Accept,Fast As a Shark,3
3,Accept,Restless and Wild,4
4,Accept,Princess of the Dawn,5
...,...,...,...
3498,Eugene Ormandy,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,3499
3499,Emerson String Quartet,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",3500
3500,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...","L'orfeo, Act 3, Sinfonia (Orchestra)",3501
3501,Nash Ensemble,"Quintet for Horn, Violin, 2 Violas, and Cello ...",3502


#### GROUP BY

- Group columns by similar values
- SELECT COUNT(id), city from students GROUP BY city

#### HAVING

- Use to apply filter AFTER a `GROUP BY` based on aggregate criteria 
- `WHERE` is applied for conditions prior to the `GROUP BY`, `HAVING` is applied afterwards

For example, if we had a table of student names and the courses they were taking, we could ask a question such as which classes have 3 or more students with the name Matt?

Such a query would look something like this:

```SQL
SELECT
  class,
  COUNT(student_name) AS number_of_alexes
FROM student_courses
WHERE student_name = "Alex"
GROUP BY 1
HAVING COUNT(student_name) >= 2;
```

In [33]:
conn = sqlite3.connect('data/tutorial.db')
c = conn.cursor()

In [34]:
# As there is no table as students changed it to responses(only table in db)
columns = [x[0] for x in c.execute('select * from responses').description]

In [35]:
columns

['submitted_ts',
 'name',
 'birthday',
 'hometown',
 'fav_food_1',
 'fav_food_2',
 'time_in_dc',
 'siblings_count']

### Questions
1. What are the names of all of the students?
2. Which student has the most siblings?
3. How many students are only children?
4. Which 3 students have lived in NYC the shortest amount of time?
5. How many students are native New Yorkers?
6. Do any two students have the same favorite food?


1. What are the names of all of the students.

In [36]:
# seected name of students from responses table
sql = '''
SELECT name FROM responses
'''
pd.read_sql_query(sql, conn)

,name
0,Stephen
1,Donna C
2,Muoyo
3,Vyjayanthi
4,Anesu Masube
5,Michael Pallante
6,Stuart Murphy
7,Darian Madere
8,Alex
9,Justin Fleury


2. Which student has the most siblings?

In [37]:
# printing maximum sibling count using max operator
sql = '''
SELECT name, max(siblings_count)
FROM responses
'''
pd.read_sql_query(sql, conn)

,name,max(siblings_count)
0,Nick,12


3. How many students are only children?

In [38]:
# printing name of students where sibling count is zero
sql = '''
SELECT count(name)
FROM responses
WHERE siblings_count = 0
'''
pd.read_sql_query(sql, conn)

,count(name)
0,0


4. Which 3 students have lived in NYC the shortest amount of time? (How long has each lived in NYC?)
    

In [39]:
# order by of column time_in_dc
sql = '''
SELECT name as Student_Name
FROM responses
ORDER BY time_in_dc
LIMIT 3
'''
pd.read_sql_query(sql, conn)

,Student_Name
0,Stephen
1,Muoyo
2,Alex


5. How many students are native New Yorkers?

In [40]:
# used like operator
sql = '''
SELECT name 
FROM responses
WHERE hometown like '%NY'
'''
pd.read_sql_query(sql, conn)

,name
0,Muoyo
1,Nimu


6. Do any two students have the same favorite food?

This problem employs the `Having` clause.  Be sure to review the difference between the where and having clause here. (Where filters apply before the group by clause and conditions following the having clause are filters applied after the group by on the resulting aggregate [statistics].) A useful example in doing so, could be to modify the question to something with an additional filtering criterion such as 'do any native new yorkers have the same favorite food?' This would force students to use a where clause prior to the group by to filter the results. Alternatively, see the question below for an alternative but related problem on favorite foods.

In [41]:
# joined the table with same table to find the favorite foods and grouped them by name
sql = '''
SELECT A.name as Student_1, B.name as Student_2
FROM responses A JOIN responses B
ON lower(A.fav_food_1) = lower(B.fav_food_1) or
   lower(A.fav_food_2) = lower(B.fav_food_1) or
   lower(A.fav_food_2) = lower(B.fav_food_2)
GROUP BY A.name, B.name
HAVING A.name != B.name
'''
pd.read_sql_query(sql, conn)

,Student_1,Student_2
0,Darian Madere,Donna C
1,Darian Madere,Jill Carrie
2,Darian Madere,Nate Lu
3,Donna C,Darian Madere
4,Donna C,Jill Carrie
5,Donna C,Nate Lu
6,Jill Carrie,Darian Madere
7,Jill Carrie,Donna C
8,Jill Carrie,Nate Lu
9,Michael Pallante,Stephen


## More Questions

What are the favorite foods of this classroom?

In [42]:
# used union operator to join (converted all fields to lower case to remove duplicates)
sql = '''
SELECT lower(fav_food_1) as FAV_FOOD
FROM responses UNION SELECT lower(fav_food_2) FROM responses
'''
pd.read_sql_query(sql, conn)

,FAV_FOOD
0,a single poached egg on a bed of arugula
1,bacon
2,beef stroganoff
3,bread
4,brownies
5,buffalo wings
6,carrot cake
7,coffee
8,crab cakes
9,dark chocolate


Which student was born closest to the cohort's graduation date (5/29/20)?

In [43]:
# used comparision to find the nearest date
sql = '''
SELECT name, birthday
FROM responses R
WHERE CAST(R.birthday as date) <= CAST('5/29' as date)
ORDER BY R.birthday desc
LIMIT 1
'''
pd.read_sql_query(sql, conn)

,name,birthday
0,Justin Fleury,5/5
